# Auto Annotation using Grounding DINO + Autodistill

- Autodistill :
    - Autodistill uses *big, slower foundation models* to train small, faster supervised models. 
    - Using autodistill, you can go from *unlabeled images* to inference with no human intervention in between.
    - As foundation models get better and better they will increasingly be able to augment or replace humans in the labeling process. 
    <img src="resource/autodistil.jpg" width="800px">
    - More about Autodistill Usage (https://docs.autodistill.com/quickstart/#step-5-label-a-dataset)
- Grounding DINO : 
    - Grounding DINO is a `self-supervised learning` algorithm that combines **DINO** (*DETR with Improved deNoising anchor boxes*) with **GLIP** (*Grounded Language-Image Pre-training*). 
    - Grounding DINO linking textual descriptions (*performed by GLIP*) to their respective visual representations (*performed by DINO*).
    - Grounding DINO can detect `arbitrary objects` with `human inputs` such as `category names` or `referring expressions`. 
    <img src="resource/grounding-dino.png" width="800px"><br><br>
    <img src="resource/model_explan1.png" width="800px"><br><br>
    <img src="resource/model_explan2.png" width="800px"><br><br>
    - More about Grounding DINO (https://github.com/IDEA-Research/GroundingDINO)
    - DINO Paper (https://arxiv.org/pdf/2203.03605)
    - GLIP Paper (https://arxiv.org/pdf/2112.03857)
    - Grounding DINO Paper (https://arxiv.org/abs/2303.05499)

## What we will do
- Setting GPU Environment
- Installing Autodistill & Grounding DINO
- Download Dataset from Google Drive Shared Link (`Dataset Scissors.zip`)
- Perform Automatic Annotation using Autodistill & Grounding DINO
- Upload Dataset & Annotation to Roboflow

⚠️⚠️⚠️ *Please open this notebook in Google Colab* by click below link ⚠️⚠️⚠️<br><br>
<a href="https://colab.research.google.com/github/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%205/5.3 auto-annotation-grounding-dino-autodistill.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 3.1 Connect GPU Environment 

- Click `Connect` button in top right Google Colab notebook,<br>
<img src="resource/cl-connect-gpu.png" width="250px">
- If connecting process completed, it will turn to something look like this<br>
<img src="resource/cl-connect-gpu-success.png" width="250px">

- Check GPU connected into Colab environment is active

In [ ]:
!nvidia-smi

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

import os
HOME = os.getcwd()
print("HOME:", HOME)

## 3.2 Installing Autodistill & Grounding DINO

In [ ]:
!pip install roboflow autodistill autodistill-grounding-dino supervision -q

## 3.3 Download Dataset from Google Drive
- We will use sample dataset [Dataset Scissors.zip](https://drive.google.com/file/d/1PFUlHCwxHAz-zMpP7UD-L47dRAzbrqWi/view) from GDrive.
- If you want to use **your own dataset**, just upload dataset to GDrive and share as **public link** in **ZIP** format.<br>
<img src="resource/gd-share.png" width="400px">
- Open the shared link in browser, and copy the `GDrive ID` in browser address bar.<br>
<img src="resource/gd-id.png" width="600px">

- Paste the GDrive ID as value `gdrive_id` variable below,

In [ ]:
!pip install gdown

import gdown

DATASET_NAME = 'Scissors_Dataset'

# default using gdrive_id Dataset `Scissors.zip` (1PFUlHCwxHAz-zMpP7UD-L47dRAzbrqWi)
gdrive_id = '1PFUlHCwxHAz-zMpP7UD-L47dRAzbrqWi' # <-----  ⚠️⚠️⚠️ USE YOUR OWN GDrive ID ⚠️⚠️⚠️

# download zip from GDrive
url = 'https://drive.google.com/uc?id=1PFUlHCwxHAz-zMpP7UD-L47dRAzbrqWi'
gdown.download(url, DATASET_NAME + ".zip", quiet=False)

# unzip dataset
!unzip -j {DATASET_NAME}.zip -d {DATASET_NAME}

## 3.4 Automatic Annotation using Autodistill & Grounding DINO

- Below, we import the dependencies required to running Autodistill & Grounding DINO

In [ ]:
from autodistill_grounding_dino import GroundingDINO
from autodistill.detection import CaptionOntology
import supervision as sv
import cv2
import os
import torch
import random

torch.use_deterministic_algorithms(False)

#### 3.4.1 Label Images with Grounding DINO

- Below, we set an *ontology*. Ontologies map `prompts` -- *text given to a model for use in labelling data* -- to the `labels` you want your dataset to include.
- For example, the ontology `"red color apple": "apple"` will send the prompt `"red color apple"` to the base model (in this example Grounding DINO). All objects matching that prompt will be labelled `"apple"`.
- Example if we have multiple object to detect (`apple`, `bottle`, `cap`): 
    ```
    {
        "red color apple": "apple"
        "milk bottle": "bottle",
        "blue cap": "cap"
    }
    ```

In [ ]:
DATASET_FOLDER = "./" + DATASET_NAME

# create ontology for `Dataset_Scissors`
# since we only want to detect `scissors` in dataset,
# just define prompt `scissors` and label `scissors`
#
ONTOLOGY = {
    "scissors": "scissors"
}

# instantiate base_model using Grounding_DINO
base_model = GroundingDINO(ontology=CaptionOntology(ONTOLOGY))

images = {}
annotations = {}

image_names = os.listdir(DATASET_FOLDER)

for image_name in image_names:

  # find full path of the image /path/to/file.jpg
  image_name = os.path.join(DATASET_FOLDER, image_name)

  # filter only image data in dataset folder
  if not image_name.endswith((".jpg", ".jpeg", ".png")):
    print(image_name)
    continue

  # perform automatic annotation (detect bounding box & label) from image in dataset folder
  predictions = base_model.predict(image_name)

  # filter only predistion results with confidence > 0.5 (50%)
  # you can change the threshold to other number preferred, e.g (0.75 for confidence above 75%)
  predictions = predictions[predictions.confidence > 0.5]

  # read image using OpenCV
  image = cv2.imread(image_name)

  # save the result into `annotations` and `images` dictionary
  annotations[image_name] = predictions
  images[image_name] = image

- Check detected boundingbox result  

In [ ]:
for image_name in image_names[:2]: # [:2] --> get only 2 sample image for checking the box result
  image_name = os.path.join(DATASET_FOLDER, image_name)

  image = images[image_name]
  detections = annotations[image_name]

  box_annotator = sv.BoxAnnotator(thickness=20)
  annotated_image = box_annotator.annotate(scene=image.copy(), detections=detections)

  %matplotlib inline
  sv.plot_image(annotated_image, (5, 5))

## 3.5 Upload Generated Annotation and Image Dataset to Roboflow


- Convert and save `annotations` data into `Pascal VOC` in `.xml` format
- The format is required by Roboflow upload API
- More abount `Pascal VOC `Format (http://host.robots.ox.ac.uk/pascal/VOC/)

In [ ]:
ANNOTATIONS_DIRECTORY = os.path.join(HOME, 'annotations')

MIN_IMAGE_AREA_PERCENTAGE = 0.002
MAX_IMAGE_AREA_PERCENTAGE = 0.80
APPROXIMATION_PERCENTAGE = 0.75

# extract class names from ONTOLOGY defined on section 3.4.1 above
CLASSES = list(ONTOLOGY.values())

sv.DetectionDataset(
    classes=CLASSES,
    images=images,
    annotations=annotations
).as_pascal_voc(
    annotations_directory_path=ANNOTATIONS_DIRECTORY,
    min_image_area_percentage=MIN_IMAGE_AREA_PERCENTAGE,
    max_image_area_percentage=MAX_IMAGE_AREA_PERCENTAGE,
    approximation_percentage=APPROXIMATION_PERCENTAGE
)

- Create **Roboflow Project** automatically using bellow script
- The upload proceess will prompting you with `Roboflow Auth Token`,<br>
<img src="resource/rb_login.png" width="500px"><br><br>
<img src="resource/rb-copy-token.png" width="800px">

In [ ]:
import roboflow
from roboflow import Roboflow

# define new project name in Roboflow
PROJECT_NAME = "scissors-auto-annotate-1"
PROJECT_DESCRIPTION = "scissors-auto-annotate-1"


roboflow.login()

workspace = Roboflow().workspace()
new_project = workspace.create_project(
    project_name=PROJECT_NAME,
    project_license="MIT",
    project_type="object-detection",
    annotation=PROJECT_DESCRIPTION)

- Upload `Image Dataset` (.jpg) and `Pascal VOC` (.xml) to newly created Roboflow Project above.

In [ ]:
import os
from tqdm.notebook import tqdm

image_paths = sv.list_files_with_extensions(directory=DATASET_FOLDER, extensions=["jpg", "jpeg", "png"])
for image_path in tqdm(image_paths):
    image_name = image_path.name
    annotation_name = f"{image_path.stem}.xml"
    image_path = str(image_path)
    annotation_path = os.path.join(ANNOTATIONS_DIRECTORY, annotation_name)
    new_project.upload(
        image_path=image_path,
        annotation_path=annotation_path,
        split="train",
        is_prediction=True,
        overwrite=True,
        tag_names=["scissors-auto-annotate"],
        batch_name="scissors-auto-annotate"
    )

- Check the uploaded dataset in Roboflow 
- Open Tab `Annotate`, click on unassigned dataset below<br>
<img src="resource/rb-auto-uploaded.png" width="800px">
- It will showing all uploaded image data with bounding box
- Just click on `Anotate Images` button in top right corner<br>
<img src="resource/rb-auto-annotated.png" width="800px">
- After that, just click `Manual Labeling` sidebar then click `Assign to Myself`<br>
<img src="resource/rb-manual-labeling.png" width="300px">
    - *Don't worrie, all data is already annotated, so nolonger need to manually annotated the data*<br><br>
- Click on `Add xx images to Dataset` button in the top right corner<br>
<img src="resource/rb-add-dataset.png" width="800px">
- Then choose method `Split Images Between Train/Valid/Test` and click `Add images` button <br>
<img src="resource/rb-train-test-split.png" width="300px">
- Completing the process like in `Pertemuan 4` to add `Preprocessing` and `Augmentation` on dataset until the `Create` process.
<img src="resource/rb-finalized.png" width="600px">
- Now the dataset ready to use, just click on `Download Dataset` button then choose the format to use like usual<br>
<img src="resource/rb-export.png" width="800px"><br><br>
- Use that Roboflow Dataset in notebook `5.1 train-yolov8-object-detection-on-custom-dataset.ipynb` by your self<br>
<a href="https://colab.research.google.com/github/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%205/5.1 train-yolov8-object-detection-on-custom-dataset.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open `5.1 train-yolov8-object-detection-on-custom-dataset.ipynb` In Colab"/></a>


